In [1]:
import torch as th
import numpy as np
import torch

import sys
if "../FFF" not in sys.path:
    sys.path.insert(0, "../FFF")

#%load_ext autoreload
#%autoreload 2

from fff_cuda import FFFLayer
from fff import FFF

Using /home/bojan/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/bojan/.cache/torch_extensions/py310_cu118/fff_cuda_/build.ninja...
Building extension module fff_cuda_...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module fff_cuda_...


In [2]:
DIM = 1024 * 8
BS = 128

fff_c = FFFLayer(DIM, DIM).cuda()
fff_l = FFF(DIM, DIM)
ff_c = th.nn.Linear(DIM, DIM, dtype=th.float).cuda()
ff = th.nn.Linear(DIM, DIM)

x_c = th.randn(BS,DIM, dtype=th.float).cuda()
x_l = x_c.clone().cpu().type(th.float)

fff_l.load_state_dict(fff_c.state_dict())

<All keys matched successfully>

In [3]:
# Check that the forward pass is the same for the cuda and the python version
th.allclose(fff_c(x_c).cpu().type(th.float32), fff_l(x_l), atol=1e-2)

True

In [4]:
%%timeit
_ = ff(x_l)

47.8 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
_ = ff_c(x_c)
torch.cuda.synchronize()

2.89 ms ± 39.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
_ = fff_l(x_l)

28.2 ms ± 909 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit
_ = fff_c(x_c)

23.3 ms ± 34.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
